In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.utils import shuffle
import string
import pandas as pd
import pickle
from sklearn.externals import joblib
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geonyoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
    return feature

In [3]:
df = pd.read_csv('./gender-classifier-DFE-791531.csv', encoding = 'latin1')
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,â°ÃÃIt felt like they were my friends and I...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
all_descriptions = df['description']
all_tweets = df['text']
all_genders = df['gender']
all_gender_confidence = df['gender:confidence']
description_tweet_gender = []

In [5]:
# Creation of bag of words for the description
bag_of_words = []
c = 0  # for the index of the row
stop = stopwords.words('english')
for tweet in all_tweets:
    description = all_descriptions[c]
    gender = all_genders[c]
    gender_confidence = all_gender_confidence[c]
    
    # remove the rows which has an empty tweet and description
    # remove the rows with unknown or empty gender
    # remove the rows which have gender:confidence < 80%
    if (str(tweet) == 'nan' and str(description) == 'nan') or str(gender) == 'nan' or str(gender) == 'unknown' or float(gender_confidence) < 0.8:
        c+=1
        continue
    
    if str(tweet) == 'nan':
        tweet = ''
    if str(description) == 'nan':
        description = ''
    
    # removal of punctuations
    for punct in string.punctuation:
        if punct in tweet:
            tweet = tweet.replace(punct, " ")
        if punct in description:
            description = description.replace(punct, " ")
    # adding the word to the bag except stopwords
    for word in tweet.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    for word in description.split():
        if word.isalpha() and word.lower() not in stop:
            bag_of_words.append(word.lower())
    
    # using tweet and description for classification
    description_tweet_gender.append((tweet+" "+description , gender))
    c += 1

In [6]:
print(description_tweet_gender[1])

('â\x80°Ã\x9bÃ\x8fIt felt like they were my friends and I was living the story with themâ\x80°Ã\x9bÂ\x9d https   t co arngE0YHNO  retired  IAN1 https   t co CIzCANPQFz I m the author of novels filled with family drama and romance ', 'male')


In [7]:
# get top 4000 words which will act as our features of each sentence
bag_of_words = nltk.FreqDist(bag_of_words)
top_words = []
for word in bag_of_words.most_common(4000):
    top_words.append(word[0])

top_words[:10]

['co', 'https', 'get', 'love', 'weather', 'like', 'http', 'one', 'life', 'new']

In [8]:
# creating the feature set, training set and the testing set
feature_set = [(find_features(top_words, text), gender) for (text, gender) in description_tweet_gender]
training_set = feature_set[:int(len(feature_set)*4/5)]
testing_set = feature_set[int(len(feature_set)*4/5):]
# print("Length of feature set", len(feature_set))
# print("Length of training set", len(training_set))
# print("Length of testing set", len(testing_set))

In [9]:
# creating a naive bayes classifier
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy = nltk.classify.accuracy(NB_classifier, testing_set)*100
print("Naive Bayes Classifier accuracy =", accuracy)

Naive Bayes Classifier accuracy = 62.843704775687414


In [11]:
NB_classifier.show_most_informative_features(10)

Most Informative Features
                 updates = True            brand : female =    168.2 : 1.0
                   dates = True            brand : female =     89.8 : 1.0
                 weather = True            brand : female =     74.8 : 1.0
                 channel = True            brand : female =     67.2 : 1.0
              continuous = True            brand : female =     59.6 : 1.0
                  update = True            brand : female =     59.2 : 1.0
                  latest = True            brand : female =     34.5 : 1.0
               subscribe = True            brand : female =     30.6 : 1.0
                  secure = True            brand : female =     30.4 : 1.0
               promoting = True            brand : female =     30.4 : 1.0


In [12]:
# creating a multinomial naive bayes classifier
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
accuracy = nltk.classify.accuracy(MNB_classifier, testing_set)*100
print("Multinomial Naive Bayes Classifier accuracy =", (accuracy))

Multinomial Naive Bayes Classifier accuracy = 64.10998552821997


In [13]:
# creating a logistic regression classifier
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
accuracy = nltk.classify.accuracy(LogisticRegression_classifier, testing_set)*100
print("Logistic Regression classifier accuracy =", accuracy)

/Users/geonyoon/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/geonyoon/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression classifier accuracy = 63.6396526772793


In [33]:
# testing with my own account
description="C, Java and Android Programmer with a very small knowledge in web development. Amateur Hacker. Engineering student for now. Huge fan of Batman."
text = "Java on being platform independent is like \" You can't C me\"."
features = find_features(top_words, description+" "+text)
print(NB_classifier.classify(features))
print(MNB_classifier.classify(features))
print(LogisticRegression_classifier.classify(features))

male
male
male


In [32]:
# Saving the Models
# Pickle allows you to serialize your python object to a file.
NB_classifier_model = pickle.dumps(NB_classifier)
MNB_classifier_model = pickle.dumps(MNB_classifier)
LogisticRegression_classifier_model = pickle.dumps(LogisticRegression_classifier)
top_words_saving = pickle.dumps(top_words)

In [31]:
# Saving the model to a file
joblib.dump(NB_classifier_model, 'NB_classifier_model.pkl')
joblib.dump(MNB_classifier_model, 'MNB_classifier_model.pkl')
joblib.dump(LogisticRegression_classifier_model, 'LogisticRegression_classifier_model.pkl')
joblib.dump(top_words_saving,'top_words_saving.pkl')

['top_words_saving.pkl']